# Data Collection

## Real-Time Data Collection:
Set up a pipeline to continuously collect real-time stock prices and other relevant financial data.
Use an API like yfinance, Alpha Vantage, or a direct market data provider to fetch real-time data.
Store this data in your Azure Blob Storage or a database for easy access and further analysis.

In [0]:
# Install necessary libraries
!pip install yfinance azure-storage-blob sqlalchemy pyodbc


Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 37.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 8.3 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.5-cp310-cp310-linux_x86_64.whl size=727736 sha256=26a8f2b96997ca427653591b8d97a8c3d34b320857e6f50dc00e5ee0b03dc177
 

## Fetch Realtime data

In [0]:
import yfinance as yf
import pandas as pd

def get_real_time_stock_price(stock_symbol):
    stock = yf.Ticker(stock_symbol)
    hist = stock.history(period='1d')
    if hist.empty:  # Check if the DataFrame is empty
        print(f"No data found for {stock_symbol}")
        return None  # Return None or appropriate value indicating no data
    else:
        return hist['Close'].iloc[-1]

def fetch_data():
    # List of stock symbols to fetch
    stocks = ['BANKBARODA.NS', 'HDFCBANK.NS', 'SBIN.NS', 'ICICIBANK.NS', 'AXISBANK.NS', '^BSESN', '^NSEI']
    data = {}
    for stock in stocks:
        price = get_real_time_stock_price(stock)
        if price is not None:  # Only add to data if price is not None
            data[stock] = price
    return data

# Fetch and display the data
data = fetch_data()
df = pd.DataFrame(data, index=[0])
print(df)  # Use print if display is not available

   BANKBARODA.NS  HDFCBANK.NS  ...        ^BSESN         ^NSEI
0     266.950012       1737.5  ...  80316.351562  24345.449219

[1 rows x 7 columns]


## upload to Blob Storage

In [0]:
from azure.storage.blob import BlobServiceClient
from datetime import datetime

# Azure Storage connection string
connect_str = 'DefaultEndpointsProtocol=https;AccountName=storageriskpredictor;AccountKey=VFB3FzSHo02JqmvdOaq2Ygr2MR5Tdq+3N/O6yTeRvr2HVysRrDK8BsmTW2u4Smp7rOBZWWD/McRO+AStGLAQzQ==;EndpointSuffix=core.windows.net'
container_name = 'riskpredict-data'

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

def upload_blob(dataframe, file_name):
    # Convert DataFrame to CSV
    csv_data = dataframe.to_csv(index=False)

    # Create a BlobClient
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)

    # Upload the CSV data
    blob_client.upload_blob(csv_data, overwrite=True)

# Generate a file name based on current timestamp and upload the DataFrame
file_name = f"realtime_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
upload_blob(df, file_name)
print(f"Data uploaded to blob storage as {file_name}")


Data uploaded to blob storage as realtime_data_20240704_045907.csv


TIll here  I was fetching real time data and storing it in a seperate csv file. But I now want to upload the realtime data in the merged model. an train my model for better prediction.

## 1. List and Read the Real-Time Data Files

In [0]:
import pandas as pd
from azure.storage.blob import BlobServiceClient
from io import StringIO

# Azure Blob Storage configuration
account_name = 'storageriskpredictor'
account_key = 'VFB3FzSHo02JqmvdOaq2Ygr2MR5Tdq+3N/O6yTeRvr2HVysRrDK8BsmTW2u4Smp7rOBZWWD/McRO+AStGLAQzQ=='
container_name = 'riskpredict-data'

# Initialize Blob Service Client
blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)

# Function to list all real-time data files
def list_blob_files(container_name):
    container_client = blob_service_client.get_container_client(container_name)
    blob_list = container_client.list_blobs()
    return [blob.name for blob in blob_list if "realtime_data_" in blob.name]

# List all real-time data files
real_time_data_files = list_blob_files(container_name)

# Load the real-time data into DataFrames
real_time_data = []
for file in real_time_data_files:
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=file)
    # Read the content of the blob as a string
    data_str = blob_client.download_blob().content_as_text()
    # Convert the string to a DataFrame
    real_time_data.append(pd.read_csv(StringIO(data_str)))

# Concatenate real-time data into a single DataFrame
real_time_data_df = pd.concat(real_time_data, ignore_index=True)


In [0]:
real_time_data_df.rename(columns={
    'BANKBARODA.NS': 'Close_BANKBARODA',
    'HDFCBANK.NS': 'Close_HDFCBANK',
    'SBIN.NS': 'Close_SBIN',
    'ICICIBANK.NS': 'Close_ICICIBANK',
    'AXISBANK.NS': 'Close_AXISBANK',
    '^BSESN': 'Close_BSE_SENSEX',
    '^NSEI': 'Close_NSEI'
}, inplace=True)

print("Renamed Real-Time Data Columns:")
print(real_time_data_df.columns)


Renamed Real-Time Data Columns:
Index(['Close_BANKBARODA', 'Close_HDFCBANK', 'Close_SBIN', 'Close_ICICIBANK',
       'Close_AXISBANK', 'Close_BSE_SENSEX', 'Close_NSEI'],
      dtype='object')


## Merge Real-Time Data with Historical Data

In [0]:
# Path to the historical dataset in Azure Blob Storage
historical_data_path = "/dbfs/mnt/riskpredict-data/merged_dataset.csv"

# Load the historical data
historical_data = pd.read_csv(historical_data_path)

# Ensure columns match and concatenate historical and real-time data
updated_data = pd.concat([historical_data, real_time_data_df], ignore_index=True)

# Drop duplicates and sort by date if necessary
updated_data.drop_duplicates(inplace=True)
updated_data.sort_values(by='Date', inplace=True)  # Replace 'date_column' with the actual date column name


In [0]:
# Concatenate historical and real-time data
updated_data = pd.concat([historical_data, real_time_data_df], ignore_index=True)

# Drop duplicates and sort by date if necessary
updated_data.drop_duplicates(inplace=True)
updated_data.sort_values(by='Date', inplace=True)  # Ensure 'Date' column is appropriately handled

print("Updated Data Preview:")
print(updated_data.head())


Updated Data Preview:
                  Date   Open  ...  Close_BSE_SENSEX    Close_NSEI
0  2019-01-02 15:30:00  283.0  ...          35891.52  23465.599609
1  2019-01-03 15:30:00  283.0  ...          35513.71  23465.599609
2  2019-01-04 15:30:00  283.0  ...          35695.10  23465.599609
3  2019-01-07 15:30:00  283.0  ...          35850.16  23465.599609
4  2019-01-08 15:30:00  283.0  ...          35980.93  23465.599609

[5 rows x 13 columns]


In [0]:
# Prepare features (X) and target (y)
X = updated_data.drop(['Date', 'Close_BANKBARODA'], axis=1)
y = updated_data['Close_BANKBARODA']

# Fill missing values with the mean of each column
X_filled = X.fillna(X.mean())

print("Features (X) preview after filling missing values:\n", X_filled.head())
print("Target (y) preview:\n", y.head())


Features (X) preview after filling missing values:
     Open   High         Low  ...  Close_AXISBANK  Close_BSE_SENSEX    Close_NSEI
0  283.0  287.5  281.600006  ...     1181.050049          35891.52  23465.599609
1  283.0  287.5  281.600006  ...     1181.050049          35513.71  23465.599609
2  283.0  287.5  281.600006  ...     1181.050049          35695.10  23465.599609
3  283.0  287.5  281.600006  ...     1181.050049          35850.16  23465.599609
4  283.0  287.5  281.600006  ...     1181.050049          35980.93  23465.599609

[5 rows x 11 columns]
Target (y) preview:
 0    286.25
1    286.25
2    286.25
3    286.25
4    286.25
Name: Close_BANKBARODA, dtype: float64


##  Train the Model with the Updated Dataset

In [0]:
# Path to save the updated dataset
updated_data_path = "/dbfs/mnt/riskpredict-data/updated_merged_dataset.csv"

# Save the updated data to DBFS
updated_data.to_csv(updated_data_path, index=False)

# Load the updated dataset for model training
model_data = pd.read_csv(updated_data_path)

Prepare X and y:
### Now, ensuring X includes all the feature columns and y is the target variable (e.g., Close_BANKBARODA).

In [0]:
# Prepare features (X) and target (y)
X = updated_data.drop(['Date', 'Close_BANKBARODA'], axis=1)
y = updated_data['Close_BANKBARODA']

# Fill missing values with the mean of each column
X_filled = X.fillna(X.mean())

print("Features (X) preview after filling missing values:\n", X_filled.head())
print("Target (y) preview:\n", y.head())


Features (X) preview after filling missing values:
     Open   High         Low  ...  Close_AXISBANK  Close_BSE_SENSEX    Close_NSEI
0  283.0  287.5  281.600006  ...     1181.050049          35891.52  23465.599609
1  283.0  287.5  281.600006  ...     1181.050049          35513.71  23465.599609
2  283.0  287.5  281.600006  ...     1181.050049          35695.10  23465.599609
3  283.0  287.5  281.600006  ...     1181.050049          35850.16  23465.599609
4  283.0  287.5  281.600006  ...     1181.050049          35980.93  23465.599609

[5 rows x 11 columns]
Target (y) preview:
 0    286.25
1    286.25
2    286.25
3    286.25
4    286.25
Name: Close_BANKBARODA, dtype: float64


In [0]:
from sklearn.model_selection import train_test_split

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_filled, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (1080, 11)
X_test shape: (270, 11)
y_train shape: (1080,)
y_test shape: (270,)


In [0]:
from sklearn.ensemble import RandomForestRegressor

# Initialize the Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)


Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

RandomForestRegressor(random_state=42)

In [0]:
from sklearn.metrics import mean_squared_error, r2_score

# Predicting on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")


Mean Squared Error: 0.0
R^2 Score: 1.0


## Update: 04-07-2024

### Automating Real-Time Data Integration and Model Training

Today, we automated the integration of real-time data and updated our Random Forest model training process. Below are the detailed steps completed:

1. **Real-Time Data Integration:**
   - Automated fetching of real-time data files from Azure Blob Storage.
   - Merged real-time data with historical data after aligning column names.
   - Handled missing values by filling them with the mean of each column.

2. **Model Training with Updated Dataset:**
   - Prepared features (X) and target variable (y) from the updated dataset.
   - Split the dataset into training and testing sets.
   - Trained a Random Forest model with the training data.
   - Evaluated the model using Mean Squared Error (MSE) and R^2 Score.

#### Model Evaluation Results:

- **Mean Squared Error:** 0.0
- **R^2 Score:** 1.0

These perfect evaluation metrics suggest that the model may be overfitting, meaning it performs exceptionally well on the training data but might not generalize well to unseen data. This will be addressed in the next steps.

3. **Job Scheduling:**
   - Scheduled a job to automate the daily process of fetching real-time data, merging it with historical data, and updating the model.




---



In [0]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_scores = cross_val_score(model, X_filled, y, cv=5, scoring='neg_mean_squared_error')

# Convert negative MSE scores to positive
cv_mse_scores = -cv_scores

print(f"Cross-Validation Mean Squared Error: {cv_mse_scores.mean()}")


Cross-Validation Mean Squared Error: 1.2447514756948308


### hyperparameter tuning

In [0]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the model
grid_search.fit(X_filled, y)

# Get the best parameters
best_params = grid_search.best_params_

print(f"Best Parameters: {best_params}")

# Train the model with the best parameters
best_model = RandomForestRegressor(**best_params)
best_model.fit(X_train, y_train)


Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}


Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

RandomForestRegressor(n_estimators=50)